In [1]:
####This notebook required run on  parallel algorithms which base on MPI####
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 
import tracemalloc

In [2]:
str_path = './TestDataSize 1k-10k/'

different_size_time_consuming =[]

csvfilename = str_path + 'test_data_1000.csv'
cal_coords_list =[]
cal_y_list =[]
cal_X_list =[]
delt_stwr_intervel =[0.0]

csvFile = open(csvfilename, "r")
#ReadData form the floder to test comsuming time 
df = pd.read_csv(csvFile,header = 0,names=['cal_coordsX','cal_coordsY','cal_x1','cal_x2','cal_y','time_stamp'],
                 dtype = {"cal_coordsX" : "float64","cal_coordsY":"float64",
                          "cal_x1":"float64","cal_x2":"float64",
                          "cal_y" : "float64",
                          "time_stamp":"float64"},
                 skip_blank_lines = True,
                 keep_default_na = False)
df.info()
df = df.sort_values(by=['time_stamp'])  
all_data = df.values
tick_time = all_data[0,-1]
cal_coord_tick = []
cal_X_tick =[]
cal_y_tick =[]
time_tol = 1.0e-7

lensdata = len(all_data)
for row in range(lensdata):
    cur_time = all_data[row,-1]
    if(abs(cur_time-tick_time)>time_tol):
        cal_coords_list.append(np.asarray(cal_coord_tick))
        cal_X_list.append(np.asarray(cal_X_tick))
        cal_y_list.append(np.asarray(cal_y_tick))
        delt_t = cur_time - tick_time
        delt_stwr_intervel.append(delt_t) 
        tick_time =cur_time
        cal_coord_tick = []
        cal_X_tick =[]
        cal_y_tick =[]
    coords_tick = np.array([all_data[row,0],all_data[row,1]])
    cal_coord_tick.append(coords_tick)
    x_tick = np.array([all_data[row,2],all_data[row,3]])
    cal_X_tick.append(x_tick)
    y_tick = np.array([all_data[row,4]])
    cal_y_tick.append(y_tick)
cal_cord_gwr = np.asarray(cal_coord_tick)
cal_X_gwr  = np.asarray(cal_X_tick)
cal_y_gwr = np.asarray(cal_y_tick)  
cal_coords_list.append(np.asarray(cal_coord_tick))
cal_X_list.append(np.asarray(cal_X_tick))
cal_y_list.append(np.asarray(cal_y_tick))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cal_coordsX  1000 non-null   float64
 1   cal_coordsY  1000 non-null   float64
 2   cal_x1       1000 non-null   float64
 3   cal_x2       1000 non-null   float64
 4   cal_y        1000 non-null   float64
 5   time_stamp   1000 non-null   float64
dtypes: float64(6)
memory usage: 54.7 KB


In [3]:
#Fast-stwr use different matrix splitting sizes 
#(1000 × 1000, 2000×2000,3000×3000,4000×4000，5000×5000,6000×6000,7000×7000,8000×8000,9000×9000 and 10000× 10000). 
mem_size_used1 = []
mem_size_peaks1 = []
mem_size_used2 = []
mem_size_peaks2 = []
splitting_sizes = [300*300,400*400,500*500,600*600,700*700,800*800,900*900,1000*1000]
#splitting_sizes = [1000*1000,2000*2000,3000*3000,4000*4000,5000*5000,6000*6000,7000*7000,8000*8000,9000*9000,10000*10000]
#splitting_sizes = [10000*10000]
for sz_mem in splitting_sizes:
    start = time.time()
    tracemalloc.start()   
    stwr_selector_ = Sel_Spt_BW(cal_coords_list, cal_y_list, cal_X_list,#gwr_bw0,
                                delt_stwr_intervel,spherical = False,max_cal_tol =sz_mem)
    #Here we use parallel algorithms.
    optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search(nproc = 12)
    current_used1, peak_1 = tracemalloc.get_traced_memory()    
   
    mem_size_used1.append(current_used1)
    mem_size_peaks1.append(peak_1)
    
    stwr_model = STWR(cal_coords_list,cal_y_list,cal_X_list,delt_stwr_intervel,
                      optsita,opt_gwr_bw0,tick_nums=opt_btticks,alpha =optalpha,spherical = False,max_cal_tol =sz_mem,recorded=1)   
    current_used2, peak_2 = tracemalloc.get_traced_memory()    
    tracemalloc.stop()
    mem_size_used2.append(current_used2)
    mem_size_peaks2.append(peak_2)
    #mem_size_used1 = sys.getsizeof(stwr_model)
    # mem_size_models.append(mem_size_model)
    end = time.time()
    comsuming = end - start
    different_size_time_consuming.append(comsuming)
    str_comsuming = '{} seconds'.format(comsuming)
    strprint ='Time-consuming of Fast-STWR when calibrating the splitting_sizes: ('+ str(sz_mem) + ' ) '+ str_comsuming
    print(strprint)
    stwr_results = stwr_model.fit()
    print(stwr_results.summary())


Time-consuming of Fast-STWR when calibrating the splitting_sizes: (90000 ) 95.83077836036682 seconds
Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                        2904912.904
Log-likelihood:                                                   -1242.147
AIC:                                                               2490.295
AICc:                                                              2492.500
BIC:                                                            2903869.136
R2:                                                                   0.014
Adj. R2:                                                              0.004

Variable                           

Time-consuming of Fast-STWR when calibrating the splitting_sizes: (360000 ) 94.78996181488037 seconds
Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                        2904912.904
Log-likelihood:                                                   -1242.147
AIC:                                                               2490.295
AICc:                                                              2492.500
BIC:                                                            2903869.136
R2:                                                                   0.014
Adj. R2:                                                              0.004

Variable                          

Time-consuming of Fast-STWR when calibrating the splitting_sizes: (810000 ) 103.18209743499756 seconds
Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                        2904912.904
Log-likelihood:                                                   -1242.147
AIC:                                                               2490.295
AICc:                                                              2492.500
BIC:                                                            2903869.136
R2:                                                                   0.014
Adj. R2:                                                              0.004

Variable                         

In [4]:
for i in range(8):
    print(f"Current memory usage is {mem_size_used1[i] / 10**6}MB; Peak was {mem_size_peaks1[i] / 10**6}MB")

Current memory usage is 0.2095MB; Peak was 0.237094MB
Current memory usage is 0.167365MB; Peak was 0.197523MB
Current memory usage is 0.167249MB; Peak was 0.205613MB
Current memory usage is 0.167249MB; Peak was 0.197379MB
Current memory usage is 0.167393MB; Peak was 0.197523MB
Current memory usage is 0.167465MB; Peak was 0.197595MB
Current memory usage is 0.151589MB; Peak was 0.182065MB
Current memory usage is 0.165401MB; Peak was 0.195531MB


In [5]:
for i in range(8):
    print(f"Current memory usage is {mem_size_used2[i] / 10**6}MB; Peak was {mem_size_peaks2[i] / 10**6}MB")

Current memory usage is 3.512001MB; Peak was 7.108542MB
Current memory usage is 3.444641MB; Peak was 8.890782MB
Current memory usage is 5.040901MB; Peak was 14.331858MB
Current memory usage is 5.035349MB; Peak was 14.330258MB
Current memory usage is 5.035621MB; Peak was 14.33053MB
Current memory usage is 5.035565MB; Peak was 14.330474MB
Current memory usage is 5.019689MB; Peak was 14.314598MB
Current memory usage is 5.034005MB; Peak was 14.328914MB


In [6]:
for i in range(8):
    print(f"runtime: {different_size_time_consuming[i] }Seconds")

runtime: 95.83077836036682Seconds
runtime: 98.25402235984802Seconds
runtime: 101.08972454071045Seconds
runtime: 94.78996181488037Seconds
runtime: 97.97885918617249Seconds
runtime: 93.41315007209778Seconds
runtime: 103.18209743499756Seconds
runtime: 106.46962785720825Seconds
